In [1]:
from lib.data import _eICUReader
from lib import data
import pandas as pd
import numpy as np

In [2]:
outcome = 'ARF'
T = 4

In [ ]:
tr_loader, va_loader, te_loader = data.get_train_val_test(outcome, duration=T, fuse=True)

Reading files


In [4]:
X_tr, s_tr, y_tr = tr_loader.dataset.X, tr_loader.dataset.s, tr_loader.dataset.y
X_va, s_va, y_va = va_loader.dataset.X, va_loader.dataset.s, va_loader.dataset.y
X_te, s_te, y_te = te_loader.dataset.X, te_loader.dataset.s, te_loader.dataset.y

In [5]:
# Concatenate tr+va to create large training set (used for cross-validation)
Xtr = np.concatenate([X_tr, X_va])
ytr = np.concatenate([y_tr, y_va]).ravel()
Str = np.concatenate([s_tr, s_va])

Xte = X_te
yte = y_te.ravel()
Ste = s_te

# Flatten time series features
Xtr = Xtr.reshape(Xtr.shape[0], -1)
Xte = Xte.reshape(Xte.shape[0], -1)

# Combine time-invariant and time series
Xtr = np.concatenate([Xtr, Str], axis=1)
Xte = np.concatenate([Xte, Ste], axis=1)

print(Xtr.shape, ytr.shape, Xte.shape, yte.shape)


(118091, 24133) (118091,) (20749, 24133) (20749,)


In [6]:
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import StratifiedShuffleSplit, StratifiedKFold, GridSearchCV, RandomizedSearchCV
from sklearn import metrics, feature_selection, utils
import scipy.stats
from joblib import Parallel, delayed
from tqdm import tqdm_notebook as tqdm

In [7]:
clf = LogisticRegression(solver='lbfgs')

In [8]:
clf.fit(Xtr, ytr)

/home/tangsp/venv/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:947: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [9]:
metrics.roc_auc_score(ytr, clf.decision_function(Xtr))

0.922122902294862

In [10]:
metrics.roc_auc_score(yte, clf.decision_function(Xte))

0.7853540245414681